In [1]:
import numpy as np
import pandas as pd
import urllib2, urllib, json
from datetime import date, timedelta, time

In [2]:
class Yahoo:
    
    def get_data(self):
        baseurl = "https://query.yahooapis.com/v1/public/yql?"
        yql_query = "select item from weather.forecast where woeid=2122265"
        yql_url = baseurl + urllib.urlencode({'q':yql_query}) + "&format=json"
        result = urllib2.urlopen(yql_url).read()
        data = json.loads(result)['query']['results']['channel']['item']
        
        self.current_temp = float(data['condition']['temp'])
        forecast = data['forecast'][1]
        self.tomorrow_temp = (float(forecast['low']), float(forecast['high']))
        
        # print forecast
    def predict(self):
        self.get_data()
        return self.tomorrow_temp[1]
        

In [3]:
class Wunderweather:
    def get_data(self):
        f = urllib2.urlopen('http://api.wunderground.com/api/d3b5ad73a85ed802/geolookup/conditions/q/ru/moscow.json')
        json_string = f.read()
        parsed_json = json.loads(json_string)
        # print parsed_json
        location = parsed_json['location']['city']
        temp_f = parsed_json['current_observation']['temp_f']
        print "Current temperature in %s is: %s" % (location, temp_f)
        self.current_temp = float(temp_f)
        
        f = urllib2.urlopen('http://api.wunderground.com/api/d3b5ad73a85ed802/forecast/q/ru/moscow.json')
        json_string = f.read()
        parsed_json = json.loads(json_string)
    
        
        tomorrow_temp = parsed_json['forecast']['simpleforecast']['forecastday'][1]
        self.tomorrow_temp = (float(tomorrow_temp['low']['fahrenheit']), float(tomorrow_temp['high']['fahrenheit']))
        
        
        yesterday = date.today() - timedelta(days=1)

        f = urllib2.urlopen('http://api.wunderground.com/api/d3b5ad73a85ed802/history_' 
                            + str(yesterday).replace('-', '')
                            + '/q/ru/moscow.json')
        json_string = f.read()
        parsed_json = json.loads(json_string)
        self.maxtemp_yesterday = float(parsed_json['history']['dailysummary'][0]['maxtempi'])
    
    def predict(self):
        self.get_data()
        return self.tomorrow_temp[1]

In [4]:
class Weather:
    yahoo = Yahoo()
    wun = Wunderweather()
    
    experts = [yahoo, wun]
    
    def predict(self):            
        return [expert.predict() for expert in experts]
    
    def calc_losses(self, predictions):
        return map(lambda x: abs(x-self.find_truth()), predictions)
    
    def find_the_truth(self):
        return self.wun.maxtemp_yesterday
    
    def save_predictions(predictions):
        with open("predictions.txt", "w") as f:
            for prediction in predictions:
                f.write("%s\n" % str(prediction))
                
    def load_old_predictions():
        with open('predictions.txt', 'r') as f:
            lines = f.readlines()
            predictions = [float(line.strip()) for line in lines]
        return predictions

In [5]:
def save_my_answer(value):
    with open("my_answers.txt", "a") as f:
        f.write(value)

def save_the_truth(value):
    with open("true_answers.txt", "a") as f:
        f.write(value)

In [6]:
def play_with_me(N):
    superman = Distributor(N)
    weather = Weather()
    
    print("Are you an adequate guy? Start this only 1 time per day, I have no time to add a verifyer")
    
    if (input == 'no'): # Wait for midday! The best time to start this infernal mechanism!
        
        predictions = experts.predict()
        
        weather.save_predictions(predictions)
        
        my_answer = superman.analyze_experts(predictions)
        
        truth = weather.find_the_truth()
        
        save_my_answer(my_answer)
        
        save_the_truth(truth)
        
        old_predictions = weather.load_old_predictions()
        
        weather.save_weights(recalc_weights(weather.calc_losses(old_predictions)))

In [7]:
class Distributor:
    BETA = 0.5
    
    def __init__(self, N):
        try:
            with open('weights.txt', 'r') as f:
                lines = f.readlines()
                self.weights =[float(line.strip()) for line in lines]
        except:
            self.weights = np.ones((1, N))[0]
    
    def recalc_weights(self, losses):
        for i in range(self.N):
            self.weights[i] *= self.BETA ** losses[i]
            
    def calc_probs(self):
        norm = sum(self.weights)
        return map(lambda x: x / norm, self.weights)
    
    def save_weights(weights):
        with open("weights.txt", "w") as f:
            for weight in weights:
                f.write("%s\n" % str(weight))

In [8]:
ya = Yahoo()
ya.get_data()
ya.current_temp, ya.tomorrow_temp

(36.0, (34.0, 60.0))

In [9]:
wun = Wunderweather()
wun.get_data()
wun.maxtemp_yesterday

Current temperature in Moscow is: 33.6


50.0